In [14]:
import json
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import pickle   
from lifelines import CoxPHFitter # install lifelines-0.30.0 if not already installed
from lifelines.utils import concordance_index


In [ ]:
# load test features 
test_radiomics = pd.read_csv('D:\\Shruti\\IS_project\\test_radiomics_lung_tumor.csv') # --> change the file per ROI

# remove features that dont have 'original' in their name
keep_cols = [col for col in test_radiomics.columns if ('diagnostics' not in col)] # texture features
# keep_cols = [col for col in keep_cols if ('diagnostics' not in col) and ('shape' in col)] # shape features only
test_radiomics = test_radiomics[keep_cols] 

# retain patient_id separately; you need the surival data for these patients also (survival time and event columns)
pids = test_radiomics['patient_id'].values
# survival_time = test_radiomics['survival_time'].values
# event = test_radiomics['event'].values
test_radiomics = test_radiomics.drop(columns=['patient_id'])

# add suffix _NSCLC to each column name
test_radiomics.columns = [col + '_NSCLC' for col in test_radiomics.columns]
test_radiomics.head()

,original_shape_Elongation_NSCLC,original_shape_Flatness_NSCLC,original_shape_LeastAxisLength_NSCLC,original_shape_MajorAxisLength_NSCLC,original_shape_Maximum2DDiameterColumn_NSCLC,original_shape_Maximum2DDiameterRow_NSCLC,original_shape_Maximum2DDiameterSlice_NSCLC,original_shape_Maximum3DDiameter_NSCLC,original_shape_MeshVolume_NSCLC,original_shape_MinorAxisLength_NSCLC,...,original_glszm_SmallAreaHighGrayLevelEmphasis_NSCLC,original_glszm_SmallAreaLowGrayLevelEmphasis_NSCLC,original_glszm_ZoneEntropy_NSCLC,original_glszm_ZonePercentage_NSCLC,original_glszm_ZoneVariance_NSCLC,original_ngtdm_Busyness_NSCLC,original_ngtdm_Coarseness_NSCLC,original_ngtdm_Complexity_NSCLC,original_ngtdm_Contrast_NSCLC,original_ngtdm_Strength_NSCLC
0,0.597984,0.278704,60.529228,217.181354,265.176304,177.179287,213.379481,281.557322,33323.424471,129.870905,...,719.978255,0.001117,6.633463,0.246669,5922.147892,1.389149,0.000165,4685.449422,0.047835,0.530341
1,0.536308,0.179125,49.928049,278.732760,217.567958,258.324451,179.751324,292.885624,8901.971397,149.486739,...,480.452011,0.003300,7.124355,0.186729,2279.536545,0.393041,0.001044,1677.072809,0.085658,1.638972
2,0.183543,0.128461,40.594035,316.002895,23.697089,65.336257,223.570926,248.480687,8135.962996,58.000006,...,555.390027,0.002030,6.796915,0.301276,270.689021,0.391701,0.001270,2416.298617,0.234645,1.433638
3,0.547651,0.347226,54.924795,158.181779,237.867159,219.905528,216.694081,239.180912,154360.623671,86.628407,...,761.890966,0.001155,6.867369,0.152001,103693.178734,6.922142,0.000039,2259.011038,0.039917,0.147094
4,0.351964,0.251468,20.596042,81.903353,242.375265,90.488367,172.598323,264.417673,11225.827258,28.827071,...,747.920343,0.001366,6.879329,0.265786,842.465312,0.409995,0.000838,2477.079569,0.073949,1.493505


In [ ]:
# load the selected features from results.json
results_json = json.load(open('D:\\Shruti\\IS_project\\models\\tumor_texture\\results.json'))
selected_features = results_json['selected_features']

# make new dataframe X_test from test_radiomics and retain patient_id_NSCLC column
X_test = test_radiomics.copy()

# filter only texture features (remove shape features)
f_names = X_test.columns.tolist()
texture_features = [f for f in f_names if 'shape' not in f]
X_test = X_test[texture_features]

# scale test data using the same scalar used for training
with open('D:\\Shruti\\IS_project\\models\\tumor_texture\\scalar.pkl', 'rb') as f:
    scaler = pickle.load(f)
X_test = pd.DataFrame(scaler.transform(X_test.values), columns=X_test.columns)  


# select only the selected features from test_radiomics
X_test_selected = X_test[selected_features]

# check if the number of selected features matches the number of columns in X_test_selected
print(len(selected_features), len(X_test_selected.columns))

37 37


c:\Users\P70073858\.conda\envs\shruti_env\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
# load CoxPH model from pickle file


with open('D:\\Shruti\\IS_project\\models\\tumor_texture\\coxph_model.pkl', 'rb') as f:
    cph = pickle.load(f)

# predict risk scores for test set
risk_scores = -cph.predict_partial_hazard(X_test_selected)

# calculate C-index --> please ensure you pass the correct survival time and event data for the test set patients here
test_c_index = concordance_index(event_observed=survival_time_column,  # --> edit as needed
                                 predicted_scores=risk_scores,
                                 event_times=event_column) # --> edit as needed
print("C-index on test set: ", test_c_index)

NameError: name 'survival_time_column' is not defined